### Comic Book list Scraper from Publisher A-Z list

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
import numpy as np
from time import sleep
import requests
from tqdm.notebook import tqdm
import time
import random
from fake_useragent import UserAgent

In [2]:
pd.options.display.max_rows = 100

In [136]:
letter = 'w'
pub_id = '3'
main_link = 'https://comicbookrealm.com/publisher/3/image-comics/w' # Normally we get this from publishers dataframe

In [137]:
ua = UserAgent()

In [138]:
headers = {'User-agent': f'{ua.random}'}
response = requests.get(main_link, headers=headers)
response.status_code

200

In [30]:
soup = BeautifulSoup(response.content, 'html.parser')

In [31]:
titles_lst = soup.find_all('td', class_='title')
titles_lst = [td.find('a') for td in titles_lst]

titles_links = [a['href'] for a in titles_lst if a['href'] is not None]

titles = [a.text for a in titles_lst]

In [32]:
volume = [td.text.strip() for td in soup.find_all('td', class_='volume')]
years = [td.text.strip() for td in soup.find_all('td', class_='years')]
issues = [td.text.strip() for td in soup.find_all('td', class_='issues data-column')]

In [3]:
def comic_list_scraper(pub_id, pub_name, letter, link, fake_agent=True):
    if fake_agent:
        ua = UserAgent()
        headers = {'User-agent': f'{ua.random}'}
    else:
        headers = {'User-agent': ''}
    
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    titles_lst = soup.find_all('td', class_='title')
    titles_lst = [td.find('a') for td in titles_lst]
    
    titles_links = [a['href'] for a in titles_lst if a['href'] is not None]
    titles = [a.text.strip() for a in titles_lst]
    volume = [td.text.strip() for td in soup.find_all('td', class_='volume')]
    years = [td.text.strip() for td in soup.find_all('td', class_='years')]
    issues = [td.text.strip() for td in soup.find_all('td', class_='issues data-column')]
    
    df = pd.DataFrame({
                        'pub_id': [pub_id]*len(titles),
                        'pub_name': [pub_name]*len(titles),
                        'letter': [letter]*len(titles),
                        'title': titles,
                        'title_link': titles_links,
                        'volume': volume,
                        'years': years,
                        'issues': issues,
        
                        })
    return df

In [4]:
# comic_list_scraper(3, 'image-comics', 'w', main_link)

In [20]:
pubs = pd.read_csv('publishers_data.csv', index_col=0)

In [21]:
site_link = 'https://comicbookrealm.com'
titles_data = []
pubs = pubs.iloc[1090:, :]
iterates = tqdm(zip(range(len(pubs.id)), pubs.id, pubs.name, pubs.titles_AZ.apply(eval)))
for i, id, name, titles in iterates:
    iterates.set_description(f"Processing: {name} - {i + 1} out of {pubs.shape[0]}")

    for title in titles:
        letter = title[-1]
        link = site_link + title[0]
        try:
            dat = comic_list_scraper(id, name, letter, link, fake_agent=False)
        except:
            continue
        titles_data.append(dat)
        time.sleep(0.1 + random.random())


0it [00:00, ?it/s]

In [22]:
df_titles = pd.concat(titles_data).reset_index(drop=True)

In [23]:
df_titles

,pub_id,pub_name,letter,title,title_link,volume,years,issues
0,2919.0,Kiddieland Books,l,Lil Keiki,/series/12297/0/kiddieland-books-lil-keiki,,2005,2
1,2920.0,Red Eagle Publishing,r,Robert Jordan's New Spring,/series/12302/0/red-eagle-publishing-robert-jo...,,2005-2006,5
2,2921.0,Timely Comics,a,Adventures of Big Boy,/series/60/0/timely-comics-adventures-of-big-b...,Vol. 1,1955-1996,536
3,2921.0,Timely Comics,a,All-Select Comics,/series/88/0/timely-comics-all-select-comics,,1943-1946,11
4,2921.0,Timely Comics,a,Amazing Comics,/series/105/0/timely-comics-amazing-comics,,1944,1
...,...,...,...,...,...,...,...,...
22432,8249.0,Vosshogg Productions,t,The Templetons,/series/68616/0/vosshogg-productions-the-templ...,One-Shot,2000,1
22433,8250.0,Centrala,j,Jarmil In India,/series/68619/0/centrala-jarmil-in-india,,2021,1
22434,8251.0,MPH Comics,p,Pop Art Funnies,/series/68622/0/mph-comics-pop-art-funnies,,2003-2007,9
22435,8252.0,A is For,c,Clinic Escorts: Stories from the Front Lines,/series/68623/0/a-is-for-clinic-escorts-storie...,,2020,1


In [24]:
df_titles.to_csv('titles_list_data-part2.csv')

In [149]:
pubs

,id,name,logo_link,titles_tracking,comics_tracking,most_valuable,most_owned,most_wanted,new_additions,titles_AZ
